In [62]:
import pandas as pd
import numpy as np

#importting data

cig_data = pd.read_csv("/Users/avanthpakanati/Desktop/ECON:HLTH Research Seminar /Homework3/data/Input/The_Tax_Burden_on_Tobacco__1970-2019_20250304.csv")
cpi_data = pd.read_excel("/Users/avanthpakanati/Desktop/ECON:HLTH Research Seminar /Homework3/data/Input/SeriesReport-20250226085315_38a6dd.xlsx", skiprows=11)



In [63]:
#cleaning tobacco data 

cig_data["measure"] = cig_data["SubMeasureDesc"].map({
    "Average Cost per pack": "cost_per_pack",
    "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
    "Federal and State tax as a Percentage of Retail Price": "tax_percent",
    "Federal and State Tax per pack": "tax_dollar",
    "Gross Cigarette Tax Revenue": "tax_revenue",
    "State Tax per pack": "tax_state"
})
cig_data = cig_data.rename(columns={
    "LocationAbbr": "state_abb", 
    "LocationDesc": "state", 
    "Data_Value": "value"
})
cig_data = cig_data[["state_abb", "state", "Year", "value", "measure"]]



In [64]:
final_data = cig_data.pivot(index=["state", "Year"], columns="measure", values="value").reset_index()
final_data = final_data.sort_values(by=["state", "Year"])



In [65]:
# cleaning CPI data
print("Columns in DataFrame after skipping rows:", cpi_data.columns.tolist()) 

cpi_data = cpi_data.melt(id_vars=["Year"], 
                          value_vars=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                          var_name="month", 
                          value_name="index")

cpi_data = cpi_data.groupby("Year", as_index=False).agg({"index": "mean"})




Columns in DataFrame after skipping rows: ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'HALF1', 'HALF2']


In [67]:
# Adjust to 2010 dollars
final_data = final_data.merge(cpi_data, on="Year", how="left")
final_data["price_cpi"] = final_data["cost_per_pack"] * (218 / final_data["index"])

In [68]:
final_data.to_csv("/Users/avanthpakanati/Desktop/ECON:HLTH Research Seminar /Homework3/data/Output/tax_data.csv")